# 3 - Million Russian Troll Tweets
- James M Irving, Ph.D.
- Mod 4 Project
- Flatiron Full Time Data Science Bootcamp - 02/2019 Cohort

## GOAL: 

- *IF I can get a control dataset* of non-Troll tweets from same time period with similar hashtags:*
    - Use NLP to predict of a tweet is from an authentic user or a Russian troll.
- *If no control tweets to compare to*
    - Use NLP to predict how many retweets a Troll tweet will get.
    - Consider both raw # of retweets, as well as a normalized # of retweets/# of followers.
        - The latter would give better indication of language's effect on propagation. 
        

## Dataset Features:
- Kaggle Dataset published by FiveThirtyEight
    - https://www.kaggle.com/fivethirtyeight/russian-troll-tweets/downloads/russian-troll-tweets.zip/2
<br>    
- Data is split into 9 .csv files
    - 'IRAhandle_tweets_1.csv' to 9

- **Variables:**
    - ~~`external_author_id` | An author account ID from Twitter~~
    - `author` | The handle sending the tweet
    - `content` | The text of the tweet
    - `region` | A region classification, as [determined by Social Studio](https://help.salesforce.com/articleView?   id=000199367&type=1)
    - `language` | The language of the tweet
    - `publish_date` | The date and time the tweet was sent
    - ~~`harvested_date` | The date and time the tweet was collected by Social Studio~~
    - `following` | The number of accounts the handle was following at the time of the tweet
    - `followers` | The number of followers the handle had at the time of the tweet
    - `updates` | The number of “update actions” on the account that authored the tweet, including tweets, retweets and likes
    - `post_type` | Indicates if the tweet was a retweet or a quote-tweet *[Whats a quote-tweet?]*
    - `account_type` | Specific account theme, as coded by Linvill and Warren
    - `retweet` | A binary indicator of whether or not the tweet is a retweet [?]
    - `account_category` | General account theme, as coded by Linvill and Warren
    - `new_june_2018` | A binary indicator of whether the handle was newly listed in June 2018
    
### **Classification of account_type**
Taken from: [rcmediafreedom.eu summary](https://www.rcmediafreedom.eu/Publications/Academic-sources/Troll-Factories-The-Internet-Research-Agency-and-State-Sponsored-Agenda-Building)

>- **They identified five categories of IRA-associated Twitter accounts, each with unique patterns of behaviors:**
    - **Right Troll**, spreading nativist and right-leaning populist messages. It supported the candidacy and Presidency of Donald Trump and denigrated the Democratic Party. It often sent divisive messages about mainstream and moderate Republicans.
    - **Left Troll**, sending socially liberal messages and discussing gender, sexual, religious, and -especially- racial identity. Many tweets seemed intentionally divisive, attacking mainstream Democratic politicians, particularly Hillary Clinton, while supporting Bernie Sanders prior to the election.
    - **News Feed**, overwhelmingly presenting themselves as U.S. local news aggregators, linking to legitimate regional news sources and tweeting about issues of local interest.
    - **Hashtag Gamer**, dedicated almost exclusively to playing hashtag games.
    - **Fearmonger**: spreading a hoax about poisoned turkeys near the 2015 Thanksgiving holiday.

>The different types of account were used differently and their efforts were conducted systematically, with different allocation when faced with different political circumstances or shifting goals. E.g.: there was a spike of activity by right and left troll accounts before the publication of John Podesta's emails by WikiLeaks. According to the authors, this activity can be characterised as “industrialized political warfare”.

___

In [1]:
import bs_ds as bs
from bs_ds.imports import *

bs_ds v. 0.7.4 ... read the docs at https://bs-ds.readthedocs.io/en/latest/index.html
For convenient loading of standard modules :
>> from bs_ds.imports import *



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Module/Package Handle
pandas,pd
numpy,np
matplotlib,mpl
matplotlib.pyplot,plt
seaborn,sns


In [2]:
import os
root_dir = 'russian-troll-tweets/'
# os.listdir('russian-troll-tweets/')
filelist = [os.path.join(root_dir,file) for file in os.listdir(root_dir) if file.endswith('.csv')]
filelist

['russian-troll-tweets/IRAhandle_tweets_1.csv',
 'russian-troll-tweets/IRAhandle_tweets_2.csv',
 'russian-troll-tweets/IRAhandle_tweets_3.csv',
 'russian-troll-tweets/IRAhandle_tweets_4.csv',
 'russian-troll-tweets/IRAhandle_tweets_5.csv',
 'russian-troll-tweets/IRAhandle_tweets_6.csv',
 'russian-troll-tweets/IRAhandle_tweets_7.csv',
 'russian-troll-tweets/IRAhandle_tweets_8.csv',
 'russian-troll-tweets/IRAhandle_tweets_9.csv']

In [3]:
# # Previewing dataset
# df = pd.read_csv(filelist[0])
# df.head(3)

## Merging full dataset

In [4]:
# Vertically concatenate 
df = pd.DataFrame()
for file in filelist:
    df_new = pd.read_csv(file)
    df = pd.concat([df,df_new], axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2973371 entries, 0 to 37554
Data columns (total 15 columns):
external_author_id    float64
author                object
content               object
region                object
language              object
publish_date          object
harvested_date        object
following             int64
followers             int64
updates               int64
post_type             object
account_type          object
new_june_2018         int64
retweet               int64
account_category      object
dtypes: float64(1), int64(5), object(9)
memory usage: 363.0+ MB


In [5]:
df.head(2)

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll


# SCRUB / EDA

In [6]:
from pandas_profiling import ProfileReport
ProfileReport(df)

Number of variables,16
Number of observations,2973371
Total Missing (%),3.5%
Total size in memory,363.0 MiB
Average record size in memory,128.0 B
Numeric,5
Categorical,9
Boolean,2
Date,0
Text (Unique),0
Rejected,0


In [ ]:
# df.info()

## Observations from Inspection / Pandas_Profiling ProfileReport

- **Language to Analyze is in `Content`:**
    - Actual tweet contents. 
 
- **Classification/Analysis Thoughts:**
    - **Variables should be considered in 2 ways:**
        - First, the tweet contents. 
            - Use NLP to engineer features to feed into deep learning.
                - Sentiment analysis, named-entity frequency/types, most-similar words. 
        - Second, the tweet metadata. 
        
### Thoughts on specific features:
- `language`
    - There are 56 unique languages. 
    - 2.4 million are English, 670 K are in Russian, etc.

### Questions to answer:
- [x] Why are so many post_types missing? (55%?)
    - Because they were added 'new_june_2018' and were not classified by the original scientists. 
- [x] How many tweets were written by a russian troll account?
    - After removing retweets, there are 1,272,848 original tweets. 
    
### Scrubing to Perform
- **Recast Columns:**
    - [ ] `publish_date` to datetime. 
- **Columns to Discard:**
    - [ ] `harvested_date` (we care about publish_date, if anything, time-wise)
    - [ ] `language`: remove all non-english tweets and drop column
    - [ ] `new_june_2018`

### Reducing Targeted Tweets to Language=English and Retweet=0 Only

- Since the goal is to use NLP to detect which tweets came from Russian trolls, we will only analyze the tweets that were originally created by a known Russian troll account

In [7]:
# Drop non-english rows
df = df.loc[df.language=='English']
df = df.loc[df.retweet==0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1272848 entries, 0 to 37431
Data columns (total 15 columns):
external_author_id    1272848 non-null float64
author                1272848 non-null object
content               1272847 non-null object
region                1271703 non-null object
language              1272848 non-null object
publish_date          1272848 non-null object
harvested_date        1272848 non-null object
following             1272848 non-null int64
followers             1272848 non-null int64
updates               1272848 non-null int64
post_type             0 non-null object
account_type          1272502 non-null object
new_june_2018         1272848 non-null int64
retweet               1272848 non-null int64
account_category      1272848 non-null object
dtypes: float64(1), int64(5), object(9)
memory usage: 155.4+ MB


In [8]:
# Drop harvested_date and new_june_2018
cols_to_drop = ['harvested_date','new_june_2018']#: remove all non-english tweets and drop column

for col in cols_to_drop:
    df.drop(col, axis=1, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1272848 entries, 0 to 37431
Data columns (total 13 columns):
external_author_id    1272848 non-null float64
author                1272848 non-null object
content               1272847 non-null object
region                1271703 non-null object
language              1272848 non-null object
publish_date          1272848 non-null object
following             1272848 non-null int64
followers             1272848 non-null int64
updates               1272848 non-null int64
post_type             0 non-null object
account_type          1272502 non-null object
retweet               1272848 non-null int64
account_category      1272848 non-null object
dtypes: float64(1), int64(4), object(8)
memory usage: 136.0+ MB


___
# Save/Load and Resume

In [ ]:
save_or_load = input('Would you like to "save" or "load" dataframe?\n("save","load","no"):')

if save_or_load.lower()=='save':
    # Save csv
    df.to_csv('russian_troll_tweets_eng_only_date_pub_index.csv')
    
if save_or_load.lower()=='load':
    import bs_ds as bs
    from bs_ds.imports import *
    # Load csva
    df = pd.read_csv('russian_troll_tweets_eng_only_date_pub_index.csv')    

In [11]:
for i in range(10):
    print(i,'\t',np.random.choice(df['content']))

0 	 Erdoğan’s referendum victory spells the end of Turkey as we know it \| Yavuz Baydar https://t.co/KEkSHkOaZP https://t.co/NnY9i7DOrf
1 	 RT 200YearsAFraud: I've had tab open in my browser for 2 weeks, intending to post it. Eerie 1890s read for FLOTUS … https://t.co/26pw6tkoKB
2 	 #Baptist Leaders Urge America #Christians To Respond To #Refugee Crisis With Faith Not Fear  https://t.co/5QMvrgbzBY https://t.co/iFwCZfarcL
3 	 #ImVotingBecause #TrumpPence identify with hard working Americans
4 	 EYEWITNESS: Vegas Murderer Regularly Verbally Abused Wife https://t.co/kbCfQft13r
5 	 Frustrated GOP Establishment is Realizing They Can’t Stop Roy Moore https://t.co/auHtfET8Vx via @truthfeednews
6 	 I wonder if Carson amuzes us once again with some new revelations #VegasGOPDebate
7 	 Man accused of hurting infant by throwing her into ceiling https://t.co/gDYWMkKC1o https://t.co/UdbG8jC84z
8 	 Thousands ‘Bike to Work’ in San Diego https://t.co/vQKxUUrqEM https://t.co/tJvTVu9w6a
9 	 Suspect Racks

### Recasting Publish date as datetime column (date_published)

In [12]:
# Recast date_published as datetime and make index
df['date_published'] = pd.to_datetime(df['publish_date'])
df.set_index('date_published', inplace=True)
print('Changed index to datetime "date_published".')

Changed index to datetime "date_published".


In [ ]:
print(df.columns)

In [ ]:
# Convert publish_date to datetime
# df['date_published'] = pd.to_datetime(df.publish_date)
print(f'Tweet dates from {np.min(df.index)}  to  {np.max(df.index)}')

### Processing Columns

In [ ]:
# Drop un-needed columns
# cols_to_drop = ['publish_date','language']
# for col in cols_to_drop:

#     df.drop(col, axis=1, inplace=True)
#     print(f'Dropped {col}.')


# # Recast categorical columns
# cols_to_cats = ['region','post_type','account_type','account_category']
# for col in cols_to_cats:

#     df[col] = df[col].astype('category')
#     print(f'Converted {col} to category.')


# # Drop problematic nan in 'contet'
# df.dropna(subset=['content'],inplace=True) # Dropping the 1 null value 

# df.head()

In [ ]:
df.info()

# Thoughts on My Search Strategy

**My Twitter API Link:**<br>
https://api.twitter.com/1.1/tweets/search/fullarchive/search.json


**Inspect Data to get search parameters:**
- [X] Get the date range for the English tweets in the original dataset<br>
    - **Tweet date range:**
        - **2012-02-06** to **2018-05-30**

- [X] Get a list of the hash tags (and their frequencies from the dataframe

**Determine most feasible and balanced well of extracting control tweets**
- [ ] How many of each tag / @'s should I try to exctract?
- [ ] what are the limitations of the API that will be a road block to getting as many tweets as desired?

In [ ]:
# Inspect Data to get search parameters:
print(f'Tweet date range:\n {min(df.index)} to {max(df.index)}')
print(f'\nTotal days: {max(df.index)-min(df.index)}')

## Determining Hashtags & @'s to search for

- Use regular expressions to extract the hashtags #words and @handles.
- Use the top X many tags as search terms for twitter API
    - There are _1,678,170 unique hashtags_ and _1,165,744 unique @'s_

In [13]:
# NEW: Make a column containing all hashtags and mentions
import re
hashtags = re.compile(r'(\#\w*)')
# df['hashtags'] = df['content'].map(lambda x: hashtags.findall(str(x)))

mentions = re.compile(r'(\@\w*)')
# df['mentions'] = df['content'].map(lambda x: mentions.findall(str(x)))

urls = re.compile(r"(http[s]?://\w*\.\w*/+\w+)")
# df['links'] = df['content'].map(lambda x: urls.findall(str(x)))

In [14]:
# Testing individual re's from above
hashtag_list = []
hashtag_list = df['content'].map(lambda x: hashtags.findall(str(x)))

In [17]:
hashtag_list[:100]

date_published
2017-10-01 19:58:00                                                 []
2017-10-01 22:43:00                                                 []
2017-10-01 23:52:00                                                 []
2017-10-01 02:47:00                                                 []
2017-10-01 02:52:00                                                 []
2017-10-01 03:47:00                                                 []
2017-10-01 03:51:00                                                 []
2017-10-01 03:58:00                                                 []
2017-10-01 04:11:00                                                 []
2017-10-10 20:57:00                                                 []
2017-10-10 21:59:00                                                 []
2017-10-10 22:06:00                                                 []
2017-10-10 22:17:00                                                 []
2017-10-10 23:42:00                                           

In [27]:
all_hashtags = []
for i in range(len(hashtag_list)):
    print(f'#{i}/{len(hashtag_list)} length={len(hashtag_list[i])}')
    
    if len(hashtag_list[i])>1:
        for x in range(len(hashtag_list[i])):
            all_hashtags.append(hashtag_list[i][x])
    else:
        all_hashtags.append(hashtag_list[i])
    
all_hashtags[:100]

#0/1272848 length=0
#1/1272848 length=0
#2/1272848 length=0
#3/1272848 length=0
#4/1272848 length=0
#5/1272848 length=0
#6/1272848 length=0
#7/1272848 length=0
#8/1272848 length=0
#9/1272848 length=0
#10/1272848 length=0
#11/1272848 length=0
#12/1272848 length=0
#13/1272848 length=0
#14/1272848 length=0
#15/1272848 length=0
#16/1272848 length=0
#17/1272848 length=0
#18/1272848 length=1
#19/1272848 length=0
#20/1272848 length=0
#21/1272848 length=0
#22/1272848 length=0
#23/1272848 length=0
#24/1272848 length=0
#25/1272848 length=0
#26/1272848 length=0
#27/1272848 length=0
#28/1272848 length=0
#29/1272848 length=0
#30/1272848 length=0
#31/1272848 length=1
#32/1272848 length=0
#33/1272848 length=0
#34/1272848 length=0
#35/1272848 length=0
#36/1272848 length=0
#37/1272848 length=0
#38/1272848 length=0
#39/1272848 length=0
#40/1272848 length=0
#41/1272848 length=0
#42/1272848 length=0
#43/1272848 length=1
#44/1272848 length=1
#45/1272848 length=1
#46/1272848 length=0
#47/1272848 length=1
#4

#379/1272848 length=1
#380/1272848 length=2
#381/1272848 length=3
#382/1272848 length=2
#383/1272848 length=0
#384/1272848 length=2
#385/1272848 length=0
#386/1272848 length=3
#387/1272848 length=1
#388/1272848 length=1
#389/1272848 length=1
#390/1272848 length=2
#391/1272848 length=0
#392/1272848 length=1
#393/1272848 length=0
#394/1272848 length=1
#395/1272848 length=2
#396/1272848 length=2
#397/1272848 length=2
#398/1272848 length=0
#399/1272848 length=0
#400/1272848 length=2
#401/1272848 length=0
#402/1272848 length=1
#403/1272848 length=1
#404/1272848 length=3
#405/1272848 length=0
#406/1272848 length=0
#407/1272848 length=2
#408/1272848 length=2
#409/1272848 length=2
#410/1272848 length=2
#411/1272848 length=0
#412/1272848 length=3
#413/1272848 length=2
#414/1272848 length=0
#415/1272848 length=2
#416/1272848 length=1
#417/1272848 length=0
#418/1272848 length=4
#419/1272848 length=2
#420/1272848 length=2
#421/1272848 length=1
#422/1272848 length=1
#423/1272848 length=1
#424/12728

#755/1272848 length=5
#756/1272848 length=1
#757/1272848 length=4
#758/1272848 length=3
#759/1272848 length=2
#760/1272848 length=1
#761/1272848 length=0
#762/1272848 length=0
#763/1272848 length=0
#764/1272848 length=1
#765/1272848 length=3
#766/1272848 length=2
#767/1272848 length=4
#768/1272848 length=4
#769/1272848 length=2
#770/1272848 length=0
#771/1272848 length=2
#772/1272848 length=1
#773/1272848 length=0
#774/1272848 length=6
#775/1272848 length=2
#776/1272848 length=3
#777/1272848 length=2
#778/1272848 length=0
#779/1272848 length=1
#780/1272848 length=1
#781/1272848 length=1
#782/1272848 length=4
#783/1272848 length=1
#784/1272848 length=2
#785/1272848 length=1
#786/1272848 length=0
#787/1272848 length=1
#788/1272848 length=7
#789/1272848 length=0
#790/1272848 length=0
#791/1272848 length=0
#792/1272848 length=4
#793/1272848 length=4
#794/1272848 length=4
#795/1272848 length=5
#796/1272848 length=1
#797/1272848 length=9
#798/1272848 length=1
#799/1272848 length=0
#800/12728

#1123/1272848 length=5
#1124/1272848 length=5
#1125/1272848 length=3
#1126/1272848 length=2
#1127/1272848 length=4
#1128/1272848 length=9
#1129/1272848 length=4
#1130/1272848 length=0
#1131/1272848 length=0
#1132/1272848 length=5
#1133/1272848 length=1
#1134/1272848 length=0
#1135/1272848 length=1
#1136/1272848 length=0
#1137/1272848 length=4
#1138/1272848 length=4
#1139/1272848 length=0
#1140/1272848 length=6
#1141/1272848 length=0
#1142/1272848 length=5
#1143/1272848 length=0
#1144/1272848 length=9
#1145/1272848 length=0
#1146/1272848 length=0
#1147/1272848 length=2
#1148/1272848 length=2
#1149/1272848 length=8
#1150/1272848 length=6
#1151/1272848 length=0
#1152/1272848 length=5
#1153/1272848 length=4
#1154/1272848 length=0
#1155/1272848 length=4
#1156/1272848 length=4
#1157/1272848 length=0
#1158/1272848 length=0
#1159/1272848 length=5
#1160/1272848 length=3
#1161/1272848 length=3
#1162/1272848 length=7
#1163/1272848 length=5
#1164/1272848 length=0
#1165/1272848 length=0
#1166/12728

#1481/1272848 length=4
#1482/1272848 length=5
#1483/1272848 length=3
#1484/1272848 length=1
#1485/1272848 length=2
#1486/1272848 length=5
#1487/1272848 length=5
#1488/1272848 length=5
#1489/1272848 length=4
#1490/1272848 length=0
#1491/1272848 length=7
#1492/1272848 length=6
#1493/1272848 length=0
#1494/1272848 length=8
#1495/1272848 length=0
#1496/1272848 length=7
#1497/1272848 length=7
#1498/1272848 length=7
#1499/1272848 length=9
#1500/1272848 length=9
#1501/1272848 length=9
#1502/1272848 length=7
#1503/1272848 length=7
#1504/1272848 length=3
#1505/1272848 length=9
#1506/1272848 length=3
#1507/1272848 length=4
#1508/1272848 length=4
#1509/1272848 length=3
#1510/1272848 length=4
#1511/1272848 length=3
#1512/1272848 length=4
#1513/1272848 length=2
#1514/1272848 length=1
#1515/1272848 length=0
#1516/1272848 length=0
#1517/1272848 length=0
#1518/1272848 length=0
#1519/1272848 length=0
#1520/1272848 length=1
#1521/1272848 length=0
#1522/1272848 length=4
#1523/1272848 length=3
#1524/12728

...

#59401/1272848 length=0
#59402/1272848 length=0
#59403/1272848 length=0
#59404/1272848 length=0
#59405/1272848 length=0
#59406/1272848 length=0
#59407/1272848 length=0
#59408/1272848 length=0
#59409/1272848 length=0
#59410/1272848 length=0
#59411/1272848 length=0
#59412/1272848 length=0
#59413/1272848 length=0
#59414/1272848 length=0
#59415/1272848 length=0
#59416/1272848 length=0
#59417/1272848 length=0
#59418/1272848 length=0
#59419/1272848 length=0
#59420/1272848 length=0
#59421/1272848 length=0
#59422/1272848 length=0
#59423/1272848 length=0
#59424/1272848 length=0
#59425/1272848 length=0
#59426/1272848 length=0
#59427/1272848 length=0
#59428/1272848 length=0
#59429/1272848 length=0
#59430/1272848 length=0
#59431/1272848 length=0
#59432/1272848 length=0
#59433/1272848 length=0
#59434/1272848 length=0
#59435/1272848 length=0
#59436/1272848 length=0
#59437/1272848 length=0
#59438/1272848 length=0
#59439/1272848 length=0
#59440/1272848 length=0
#59441/1272848 length=0
#59442/1272848 l

#59748/1272848 length=0
#59749/1272848 length=0
#59750/1272848 length=0
#59751/1272848 length=0
#59752/1272848 length=0
#59753/1272848 length=0
#59754/1272848 length=0
#59755/1272848 length=0
#59756/1272848 length=0
#59757/1272848 length=0
#59758/1272848 length=0
#59759/1272848 length=0
#59760/1272848 length=0
#59761/1272848 length=0
#59762/1272848 length=0
#59763/1272848 length=0
#59764/1272848 length=0
#59765/1272848 length=0
#59766/1272848 length=0
#59767/1272848 length=0
#59768/1272848 length=0
#59769/1272848 length=0
#59770/1272848 length=0
#59771/1272848 length=0
#59772/1272848 length=0
#59773/1272848 length=0
#59774/1272848 length=0
#59775/1272848 length=0
#59776/1272848 length=0
#59777/1272848 length=0
#59778/1272848 length=0
#59779/1272848 length=0
#59780/1272848 length=0
#59781/1272848 length=0
#59782/1272848 length=0
#59783/1272848 length=0
#59784/1272848 length=0
#59785/1272848 length=0
#59786/1272848 length=0
#59787/1272848 length=0
#59788/1272848 length=0
#59789/1272848 l

#60093/1272848 length=0
#60094/1272848 length=0
#60095/1272848 length=0
#60096/1272848 length=0
#60097/1272848 length=0
#60098/1272848 length=0
#60099/1272848 length=0
#60100/1272848 length=0
#60101/1272848 length=0
#60102/1272848 length=0
#60103/1272848 length=0
#60104/1272848 length=0
#60105/1272848 length=0
#60106/1272848 length=0
#60107/1272848 length=0
#60108/1272848 length=1
#60109/1272848 length=0
#60110/1272848 length=1
#60111/1272848 length=0
#60112/1272848 length=0
#60113/1272848 length=0
#60114/1272848 length=0
#60115/1272848 length=0
#60116/1272848 length=0
#60117/1272848 length=0
#60118/1272848 length=0
#60119/1272848 length=0
#60120/1272848 length=0
#60121/1272848 length=0
#60122/1272848 length=0
#60123/1272848 length=0
#60124/1272848 length=0
#60125/1272848 length=0
#60126/1272848 length=0
#60127/1272848 length=0
#60128/1272848 length=0
#60129/1272848 length=0
#60130/1272848 length=0
#60131/1272848 length=0
#60132/1272848 length=0
#60133/1272848 length=0
#60134/1272848 l

#60440/1272848 length=0
#60441/1272848 length=0
#60442/1272848 length=0
#60443/1272848 length=0
#60444/1272848 length=0
#60445/1272848 length=0
#60446/1272848 length=0
#60447/1272848 length=0
#60448/1272848 length=0
#60449/1272848 length=0
#60450/1272848 length=0
#60451/1272848 length=0
#60452/1272848 length=0
#60453/1272848 length=0
#60454/1272848 length=0
#60455/1272848 length=0
#60456/1272848 length=0
#60457/1272848 length=0
#60458/1272848 length=0
#60459/1272848 length=0
#60460/1272848 length=0
#60461/1272848 length=0
#60462/1272848 length=0
#60463/1272848 length=0
#60464/1272848 length=0
#60465/1272848 length=0
#60466/1272848 length=0
#60467/1272848 length=0
#60468/1272848 length=0
#60469/1272848 length=0
#60470/1272848 length=0
#60471/1272848 length=0
#60472/1272848 length=0
#60473/1272848 length=0
#60474/1272848 length=0
#60475/1272848 length=0
#60476/1272848 length=0
#60477/1272848 length=0
#60478/1272848 length=0
#60479/1272848 length=0
#60480/1272848 length=0
#60481/1272848 l

KeyboardInterrupt: 

In [ ]:
hashtag_counts = pd.Series(all_hashtags)
hashtag_counts.value_counts()

In [ ]:
len(df),len(hashtag_list)

# BOOKMARK 06/03

### def get_tags_ats

In [ ]:
# Define get_tags_ats to accept a list of text entries and return all found tags and ats as 2 series/lists
def get_tags_ats(text_to_search,exp_tag = r'(#\w*)',exp_at = r'(@\w*)', output='series',show_counts=False):
    """Accepts a list of text entries to search, and a regex for tags, and a regex for @'s.
    Joins all entries in the list of text and then re.findsall() for both expressions.
    Returns a series of found_tags and a series of found_ats.'"""
    import re
    
    # Create a single long joined-list of strings
    text_to_search_combined = ' '.join(text_to_search)
        
    # print(len(text_to_search_combined), len(text_to_search_list))
    found_tags = re.findall(exp_tag, text_to_search_combined)
    found_ats = re.findall(exp_at, text_to_search_combined)
    
    if output.lower() == 'series':
        found_tags = pd.Series(found_tags, name='tags')
        found_ats = pd.Series(found_ats, name='ats')
        
        if show_counts==True:
            print(f'\t{found_tags.name}:\n{tweet_tags.value_counts()} \n\n\t{found_ats.name}:\n{tweet_ats.value_counts()}')
                
    if (output.lower() != 'series') & (show_counts==True):
        raise Exception('output must be set to "series" in order to show_counts')
                       
    return found_tags, found_ats

In [ ]:
# Need to get a list of hash tags.
text_to_search_list = []

for i in range(len(df)):    
    tweet_contents =df['content'].iloc[i]
    text_to_search_list.append(tweet_contents)

text_to_search_list[:2]

In [ ]:
# Get all tweet tags and @'s from text_to_search_list
tweet_tags, tweet_ats = get_tags_ats(text_to_search_list, show_counts=False)

print(f"There were {len(tweet_tags)} unique hashtags and {len(tweet_ats)} unique @'s\n")

# Create a dataframe with top_tags
df_top_tags = pd.DataFrame(tweet_tags.value_counts()[:40])#,'\n')
df_top_tags['% Total'] = (df_top_tags['tags']/len(tweet_tags)*100)

# Create a dataframe with top_ats
df_top_ats = pd.DataFrame(tweet_ats.value_counts()[:40])
df_top_ats['% Total'] = (df_top_ats['ats']/len(tweet_ats)*100)

# Display top tags and ats
# bs.display_side_by_side(df_top_tags,df_top_ats)

### Notes on Top Tags and Ats:


In [ ]:
# Choose list of top tags to use in search
list_top_30_tags = df_top_tags.index[:30]
list_top_30_tags

In [ ]:
# Choose list of top tags to use in search
list_top_30_ats = df_top_ats.index[:30]
list_top_30_ats

- The most common tags include some very generic categories that may not be helpful in extracting control tweets.
    - ~~Exclude: '#news','#sports','#politics','#world','#local','#TopNews','#health','#business','#tech',~~
    - On second thought, this is entirely appropriate, since these tags would be what appears in the wild.
    - Additionally, using a larger number of them (like 30, starts to provide more targeted hashtags.<br><br>
  
- **The most common @'s are much more revealing and helpful in narrowing the focus of the results.**

___

# Using the Twitter Search API to Extract Control Tweets

- [x] Required API key are saved in the Main folder in which this repo is saved. 
- [x] Check the [Premium account docs for search syntax](https://developer.twitter.com/en/docs/tweets/search/guides/premium-operators.html)
- [x] [Check this article for using Tweepy for most efficient twitter api extraction](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./)

**LINK TO PREMIUM SEARCH API GUIDE**<br>
https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search

**Available search operators**
- Premium search API supports rules with up to 1,024 characters. The Search Tweets APIs support the premium operators listed below. See our Premium operators guide for more details.

- The base URI for the premium search API is https://api.twitter.com/1.1/tweets/search/.

**Matching on Tweet contents:**
- keyword , "quoted phrase" , # , @, url , lang


## Using tweepy to access twitter API

- [Helpful tutorial on _most efficient_ way to access twitter API](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./)

### def connect_twitter_api, def search_twitter_api

In [ ]:
# Initialzie Tweepy with Authorization Keys    
def connect_twitter_api(api_key, api_secret_key):
    import tweepy, sys
    auth = tweepy.AppAuthHandler(api_key, api_secret_key)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

    if (not api):
        print("Can't authenticate.")
        sys.exit(-1)
    return api

In [ ]:
def search_twitter_api(api_object, searchQuery, maxTweets, fName, tweetsPerQry=100, max_id=0, sinceId=None):
    """Take an authenticated tweepy api_object, a search queary, max# of tweets to retreive, a desintation filename.
    Uses tweept.api.search for the searchQuery until maxTweets is reached, saved harvest tweets to fName."""
    import sys, jsonpickle, os
    api = api_object
    tweetCount = 0
    print(f'Downloading max{maxTweets} for {searchQuery}...')
    with open(fName, 'a+') as f:
        while tweetCount < maxTweets:

            try:
                if (max_id <=0):
                    if (not sinceId):
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended')
                    else:
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry, since_id=sinceId, tweet_mode='extended')

                else:
                    if (not sinceId):
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry, max_id=str(max_id-1), tweet_mode='extended')
                    else:
                        new_tweets = api.search(q=searchQuery, count=tweetsPerQry, max_id=str(max_id-1),since_id=sinceId, tweet_mode='extended')

                if not new_tweets:
                    print('No more tweets found')
                    break

                for tweet in new_tweets:
                    f.write(jsonpickle.encode(tweet._json, unpicklable=False)+'\n')

                tweetCount+=len(new_tweets)

                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("some error : " + str(e))
                break
    print ("Downloaded {0} tweets, Saved to {1}\n".format(tweetCount, fName))

## Connect to Twitter and Harvest Tweets

### Making lists of tags and ats to query

In [ ]:
print(df.index

In [ ]:
# df_top_ats.ats[:20], df_top_tags.tags[:20]

In [ ]:
# Figure out the # of each @ and each # that i want ot query, then make a query_dict to feed into the cell below
query_ats = tuple(zip(df_top_ats.index, df_top_ats['ats']))
query_tags = tuple(zip(df_top_tags.index, df_top_tags['tags']))

# Calculate how many tweets are represented by the top 30 tags and top 30 @'s 
sum_top_tweet_tags = df_top_tags['tags'].sum()
sum_top_tweet_ats = df_top_ats['ats'].sum()
print(f"Sum of top tags = {sum_top_tweet_tags}\nSum of top @'s = {sum_top_tweet_ats}")

In [ ]:
print(query_ats[:10],'\n')
print(query_tags[:10])

In [ ]:
len(query_ats)

In [ ]:
np.sum([x[1] for x in query_ats])

In [ ]:
# Inspect Data to get search parameters:
print(f'Tweet date range:\n {min(df.index)} to {max(df.index)}')
print(f'\nTotal days: {max(df.index)-min(df.index)}')

### Reconsidering a More Thoughtful Approach to Top-Tags/@'s to Retreive
- using the most popular tags will become an issue when searching historical log of tweets.
- I am unable to query twitter for specifics dates, which means that I will have to recursively search/paginate all the way back to 05-30-2018 before saving tweets. 
    - **The API limitations mean that this will take MUCH longer for the MOST COMMON hashtags.**

- **-New Decision: Use only @'s to determine the body of tweets to use as a control.**

In [ ]:
PAUSE

### Connecting to twitter api and searching for lists of queries

In [ ]:
# Import API keys from text files (so not displayed here and not in repo)
with open('../consumer_API_key.txt','r') as f:
    api_key =  f.read()
with open('../consumer_API_secret_key.txt','r') as f:
    api_secret_key  = f.read()

#### Test searches

In [ ]:
# Manually connecting to API and doing test searches. 
import tweepy, sys
auth = tweepy.AppAuthHandler(api_key, api_secret_key)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print("Can't authenticate.")
    sys.exit(-1)

In [ ]:
# Search for a batch of test results
searchQuery='#politics'
tweetsPerQry=100

new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended')
type(new_tweets)

In [ ]:
#  Display time range of new_tweets so i can define a timetrange to test
test_dates = [x.created_at for x in new_tweets]
print(f'Range:{min(test_dates)} to {max(test_dates)}')
test_dates[0], test_dates[-1]

In [ ]:
from datetime import datetime
end_time = datetime(2019,6,2,20,0,0)
end_time

In [ ]:
## DEFINING A NEW FUNCTION TO EXAMINE THE NEW_TWEETS OUTPUTS
def check_tweet_daterange(new_tweets,timerange_begin,timerange_end,verbose=0):
    """Examines specific information for each tweet in a tweepy searchResults object."""
    
    time_start = timerange_begin
    time_end = timerange_end
    
    # Pull out each tweet's status object. 
    idx_keep_tweets = []
    for i,tweet in enumerate(new_tweets):
        if (tweet.created_at > time_start) and (tweet.created_at < time_end):
            idx_keep_tweets.append(i)
            if verbose>0:
                print(f'tweet({i} kept:{tweet.created_at})')
    return idx_keep_tweets

In [ ]:
# Determining search criteria to limit twitter results to
latest_date = max(df.index) # Get latest date from troll tweets
earliest_date = min(df.index) # Get the earliest date from troll tweets

# Convert pandas timestamps to datetime object for tweet results
latest_datetime = latest_date.to_pydatetime()
earliest_datetime = earliest_date.to_pydatetime()

In [ ]:
# def search_twitter_api_for_timerange(api_object, searchQuery, timerange_start, timerange_end, maxTweets, fName, tweetsPerQry=100, max_id=0, sinceId=None):
#     """Take an authenticated tweepy api_object, a search queary, a datetime object for earliest date and latest date,
#     max# of tweets to retreive, a desintation filename.
#     Uses tweept.api.search for the searchQuery until maxTweets is reached, saved harvest tweets to fName."""
#     import sys, jsonpickle, os
#     api = api_object
#     tweetCount = 0
#     print(f'Downloading max{maxTweets} for {searchQuery}...')
#     with open(fName, 'a+') as f:
#         while tweetCount < maxTweets:

#             try:
#                 if (max_id <=0):
#                     if (not sinceId):
#                         new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended')
#                     else:
#                         new_tweets = api.search(q=searchQuery, count=tweetsPerQry, since_id=sinceId, tweet_mode='extended')

#                 else:
#                     if (not sinceId):
#                         new_tweets = api.search(q=searchQuery, count=tweetsPerQry, max_id=str(max_id-1), tweet_mode='extended')
#                     else:
#                         new_tweets = api.search(q=searchQuery, count=tweetsPerQry, max_id=str(max_id-1),since_id=sinceId, tweet_mode='extended')

#                 if not new_tweets:
#                     print('No more tweets found')
#                     break
                
#                 max_id = new_tweets[-1].id
#                 # Insert check for timerange
#                 new_tweets_idx_keep = check_tweet_daterange(new_tweets,test_dates[-1],end_time)
#                 kept_new_tweets = [new_tweets[i] for i in new_tweets_idx_keep]

                    
#                 for tweet in kept_new_tweets:
#                     f.write(jsonpickle.encode(tweet._json, unpicklable=False)+'\n')

#                 tweetCount+=len(kept_new_tweets)

#                 print("Downloaded {0} tweets".format(tweetCount))

#             except tweepy.TweepError as e:
#                 # Just exit if any error
#                 print("some error : " + str(e))
#                 break
#     print ("Downloaded {0} tweets, Saved to {1}\n".format(tweetCount, fName))

In [ ]:
# # Get index of tweets in time range
# new_tweets_idx_keep = check_tweet_daterange(new_tweets,test_dates[-1],end_time)

# kept_new_tweets = [new_tweets[i] for i in new_tweets_idx_keep]
# kept_new_tweets[0]

#### Automated Searches:

In [ ]:
pause

In [ ]:
api = connect_twitter_api(api_key,api_secret_key)

In [ ]:
# Extract tweets for top @'s, while matching the distribution of top @'s

final_query_list = query_ats
filename = 'tweets_for_top40_ats.txt'

for q in final_query_list:
    searchQuery = q[0]
    maxTweets = q[1]
    print(f'Query={searchQuery}, max={maxTweets}')
    search_twitter_api(api, searchQuery, maxTweets, fName=filename)

In [ ]:
# # Extract tweets for top #
# final_query_list = query_tags[:3]
# filename = 'tweets_for_top3_tags.txt'

# for q in final_query_list:
#     searchQuery = q[0]
#     maxTweets = q[1]
#     print(f'Query={searchQuery}, max={maxTweets}')
#     search_twitter_api(api, searchQuery, maxTweets, fName=filename)

___

# BOOKMARK: Revisiting Quoted Tweets

In [2]:
from bs_ds.imports import *

bs_ds v. 0.7.4 ... read the docs at https://bs-ds.readthedocs.io/en/latest/index.html
For convenient loading of standard modules :
>> from bs_ds.imports import *



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\james\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Module/Package Handle
pandas,pd
numpy,np
matplotlib,mpl
matplotlib.pyplot,plt
seaborn,sns


In [3]:
df_ats = pd.read_json('tweets_for_top40_ats.txt', lines=True)

In [4]:
df_ats.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user,withheld_in_countries
0,NaN,None,2019-06-02 18:34:59,"[0, 202]","{'hashtags': [{'indices': [48, 66], 'text': 'd...",NaN,0,False,@realDonaldTrump it’s perfectly reasonable tha...,None,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
1,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @BelkissObadia: BREAKING NEWS: \n\n@realDon...,None,...,NaN,NaN,NaN,950,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
2,NaN,None,2019-06-02 18:34:59,"[17, 207]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,@realDonaldTrump I thought you were supposed t...,None,...,NaN,NaN,NaN,0,False,NaN,"<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
3,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @realDonaldTrump: Mexico is sending a big d...,None,...,NaN,NaN,NaN,3128,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
4,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @realDonaldTrump: Mexico is sending a big d...,None,...,NaN,NaN,NaN,3128,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN


In [7]:
df_ats['entities'][1]

{'hashtags': [],
 'symbols': [],
 'urls': [],
 'user_mentions': [{'id': 46085118,
   'id_str': '46085118',
   'indices': [3, 17],
   'name': 'Belkiss Obadia',
   'screen_name': 'BelkissObadia'},
  {'id': 25073877,
   'id_str': '25073877',
   'indices': [36, 52],
   'name': 'Donald J. Trump',
   'screen_name': 'realDonaldTrump'}]}

In [12]:
df = df_ats.loc[df_ats['full_text'].str.contains('RT')]

In [13]:
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,truncated,user,withheld_in_countries
1,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @BelkissObadia: BREAKING NEWS: \n\n@realDon...,None,...,NaN,NaN,NaN,950,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
3,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @realDonaldTrump: Mexico is sending a big d...,None,...,NaN,NaN,NaN,3128,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/#!/download/ipad"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
4,NaN,None,2019-06-02 18:34:59,"[0, 139]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @realDonaldTrump: Mexico is sending a big d...,None,...,NaN,NaN,NaN,3128,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
6,NaN,None,2019-06-02 18:34:59,"[0, 122]","{'hashtags': [], 'symbols': [], 'urls': [{'dis...",NaN,0,False,RT @TomFitton: Obama CIA Chief John Brennan wa...,None,...,NaN,NaN,NaN,2412,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN
10,NaN,None,2019-06-02 18:34:59,"[0, 140]","{'hashtags': [], 'symbols': [], 'urls': [], 'u...",NaN,0,False,RT @2runtherace: 1. Who is the Witch @realDona...,None,...,NaN,NaN,NaN,20,False,"{'contributors': None, 'coordinates': None, 'c...","<a href=""http://twitter.com/download/android"" ...",False,"{'contributors_enabled': False, 'created_at': ...",NaN


In [28]:
import re
re_RT=re.compile('RT \@\w*\:')

In [35]:
example_rt_tweets = df['full_text'][:10]
example_rt_tweets

1     RT @BelkissObadia: BREAKING NEWS: \n\n@realDon...
3     RT @realDonaldTrump: Mexico is sending a big d...
4     RT @realDonaldTrump: Mexico is sending a big d...
6     RT @TomFitton: Obama CIA Chief John Brennan wa...
10    RT @2runtherace: 1. Who is the Witch @realDona...
12    RT @GOPChairwoman: The abuse of government sur...
13    RT @realDonaldTrump: People have been saying f...
15    RT @realDonaldTrump: People have been saying f...
18    RT @realDonaldTrump: Mexico is sending a big d...
20    RT @frfrankpavone: FoxNews report, “Meghan Mar...
Name: full_text, dtype: object

In [36]:
type(example_rt_tweets)

pandas.core.series.Series

In [38]:
example_rt_tweets.apply(lambda x: re_RT.sub('',x))

1      BREAKING NEWS: \n\n@realDonaldTrump Trump won...
3      Mexico is sending a big delegation to talk ab...
4      Mexico is sending a big delegation to talk ab...
6      Obama CIA Chief John Brennan was ‘Key Ringlea...
10     1. Who is the Witch @realDonaldTrump is refer...
12     The abuse of government surveillance to spy o...
13     People have been saying for years that we sho...
15     People have been saying for years that we sho...
18     Mexico is sending a big delegation to talk ab...
20     FoxNews report, “Meghan Markle and Prince Har...
Name: full_text, dtype: object

In [20]:
quote_list = (['idx','quoted_status'])
for k,v in df['quoted_status'].items()[:10]:
    print
    quote_list.append([k,v])

In [22]:
import bs_ds as bs
# bs.list2df(quote_list)

TypeError: Index(...) must be called with a collection of some kind, 'idx' was passed

In [14]:
df['retweeted_status'][1]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jun 01 20:49:31 +0000 2019',
 'display_text_range': [0, 204],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'mirror.co.uk/news/politics/…',
    'expanded_url': 'https://www.mirror.co.uk/news/politics/donald-trump-wont-allowed-stay-14512697?utm_source=twitter.com&utm_medium=social&utm_campaign=sharebar',
    'indices': [155, 178],
    'url': 'https://t.co/veTq60rpfF'}],
  'user_mentions': [{'id': 25073877,
    'id_str': '25073877',
    'indices': [17, 33],
    'name': 'Donald J. Trump',
    'screen_name': 'realDonaldTrump'},
   {'id': 813286,
    'id_str': '813286',
    'indices': [141, 153],
    'name': 'Barack Obama',
    'screen_name': 'BarackObama'}]},
 'favorite_count': 3084,
 'favorited': False,
 'full_text': "BREAKING NEWS: \n\n@realDonaldTrump Trump won't be allowed to stay at Buckingham Palace during UK state visit even though President Obama did. @BarackObama \nhttps://t.co/veTq60rpfF 

In [5]:
df_ats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44127 entries, 0 to 44126
Data columns (total 32 columns):
contributors                 0 non-null float64
coordinates                  6 non-null object
created_at                   44127 non-null datetime64[ns]
display_text_range           44127 non-null object
entities                     44127 non-null object
extended_entities            4875 non-null object
favorite_count               44127 non-null int64
favorited                    44127 non-null bool
full_text                    44127 non-null object
geo                          6 non-null object
id                           44127 non-null int64
id_str                       44127 non-null int64
in_reply_to_screen_name      20622 non-null object
in_reply_to_status_id        19352 non-null float64
in_reply_to_status_id_str    19352 non-null float64
in_reply_to_user_id          20622 non-null float64
in_reply_to_user_id_str      20622 non-null float64
is_quote_status              

In [ ]:

# df_export['external_author_id'] = df_ats.loc[index=['user']['id']
# df_export['author'] = df_ats['user']['screen_name']
# df_export['content'] = df_ats['full_text']

# df_export['region'] = df_ats['user']['location']

# df_export['following'] = df_ats['user']['following']
# df_export['followers'] = df_ats['user']['followers_count']
# df_export['updates'] = np.nan
# df_export['post_type'] = 'control'
# df_export['account_type'] = 'control'
# df_export['retweet'] = df_ats['retweeted']
# df_export['account_category'] = 'control' 
# df_export['publish_date'] = df_ats['created_at']
# df_export['date_published'] = pd.to_datetime(df_ats['created_at'])
# df_export.set_index('date_published',inplace=True)
# df_export['language'] = df_ats['lang']

In [ ]:
print(df.columns)

### Notes on Making New Extracted Tweets Match Russian Troll Tweet Database

- Columns to be renamed/reformatted to match troll tweets:
    - created_at -> 'date_published'-> index
    - full_text -> 'content'
    - df['user'].
        - .['followers_count'] -> 'following'
        - .['followers_count'] -> 'followers'
        - .['screen_name'] -> 
        - .['id'] ->
- Columns missing from original troll tweets (to be removed).
    -coordinates, favorited, favorite_count, display_text_range, withheld_in_countries
    

In [ ]:
df_ats['lang'][1]

In [ ]:
df_ats.user[10]

In [ ]:
df_ats.entities[0]

In [ ]:
print(df_ats['user'][1]['location'])

In [ ]:
print(df_ats['user'][0]['id'])
print(df_ats['user'][0]['screen_name'])
print(df_ats['user'][0]['followers_count'])
print(df_ats['user'][0]['following'])

In [ ]:
df_test=pd.DataFrame()

In [ ]:
idx_row = df_ats.index[2]
curr_row = df_ats.loc[df_ats.index==idx_row]
curr_author = curr_row['user']
curr_author

In [ ]:
# df_test.loc[column==['new_col'],index==idx_row] = curr_row['user']
list(df.columns)

In [ ]:
df_columns_list =['external_author_id', 'author', 'content', 'region', 'following', 'followers', 'updates', 'post_type',
 'account_type', 'retweet', 'account_category']
df_export = pd.DataFrame(columns=df_columns_list)

In [ ]:
df_export.loc[0,'external_author_id']='test'
df_export

In [ ]:
curr_author[0]

In [ ]:
full_text=curr_row['ret']
full_text

In [ ]:
curr_row['user']
row

In [ ]:
df_export=pd.DataFrame()
df_columns_list =['external_author_id', 'author', 'content', 'region', 'following', 'followers', 'updates', 'post_type',
 'account_type', 'retweet', 'account_category']

df_export = pd.DataFrame(columns=df_columns_list)

for row in df_ats.index:
    
    curr_row = df_ats.loc[df_ats.index==row]
    curr_author = curr_row['user'][row]
    external_author_id = curr_author['id']
    author =  curr_author['screen_name']
    following = curr_author['following']
    followers = curr_author['followers_count']
    region = curr_author['location']
    full_text = curr_row['full_text'][row]
    
    df_export.loc[row, 'external_author_id'] = external_author_id
    df_export.loc[row, 'author'] = author
    df_export.loc[row, 'content'] = full_text
    df_export.loc[row, 'region'] = region
    df_export.loc[row, 'following'] = following
    df_export.loc[row, 'followers'] = followers
    df_export.loc[row, 'updates'] = np.nan
    df_export.loc[row, 'post_type'] = 'control'
    df_export.loc[row, 'account_type'] = 'control'
    df_export.loc[row, 'retweet'] = curr_row['retweeted'][row]
    df_export.loc[row, 'account_category'] = 'control' 
    df_export.loc[row, 'publish_date'] = curr_row['created_at'][row]
    df_export.loc[row, 'language'] = curr_row['lang'][row]

In [ ]:
df_export.head()

In [ ]:
df_export.to_csv('newly_extracted_control_tweets.csv')

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(df_export)

In [ ]:
df_export.['date_published'] = pd.to_datetime(df_export['publish_date'])    
df_export.set_index('date_published',inplace=True)

In [ ]:
df.info()

# Moving on to NLP Analysis while dataset is reconsidered

In [ ]:
df_data = pd.DataFrame()
for at in query_ats:
    df_data

In [ ]:
# First, get all text from original df and new top 40 ats into a dataframe (or list?)
# only need content and label
df_data = 